In [43]:
import os
import sys
from glob import glob
import h5py
import time
import progressbar

import numpy as np

from skimage import io, color, exposure, transform
from sklearn.model_selection import train_test_split

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, model_from_json
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.optimizers import SGD
from keras.utils import np_utils
from keras.callbacks import LearningRateScheduler, ModelCheckpoint
from keras import backend
backend.set_image_data_format('channels_first')

from matplotlib import pyplot
%matplotlib inline

In [18]:
NUM_CLASSES = 43
IMG_SIZE = 48

<h2> Preprocessing </h2>

1. Histogram normalization in HSV yellow
2. Crop central region
3. Resize
4. Roll RGB axis to 0

In [8]:
def preprocess(image):
    
    # Histogram normalization in HSV yellow
    temp = color.rgb2hsv(image)
    temp[:,:,2] = exposure.equalize_hist(temp[:,:,2])
    image = color.hsv2rgb(temp)
    
    # Crop central region
    ms = min(image.shape[:-1])
    center = image.shape[0]//2, image.shape[1]//2
    image = image[
        center[0] - ms//2 : center[0] + ms//2,
        center[1] - ms//2 : center[1] + ms//2,
        :
    ]
    
    # Resize
    image = transform.resize(image, (IMG_SIZE, IMG_SIZE))
    
    # Roll RGB axis to 0
    image = np.rollaxis(image, -1)
    
    return image

<h2> Preprocessing </h2>

1. Store images into numpy arrays
2. Get labels
3. Convert to one-hot

In [ ]:
root = 'GTSRB/Final_Training/Images/'
images = []
labels = []
paths = glob(os.path.join(root, '*/*.ppm'))
np.random.shuffle(paths)
total = len(paths)

start = time.time()
print("Total training images:\t{}".format(total))
for i in range(total):
    sys.stdout.write('\r')
    sys.stdout.write("Processing image\t{} | {}% complete".format(i, round(i*100/total, 2)))
    sys.stdout.flush()

    path = paths[i]
    image = preprocess(io.imread(path))
    label = int(path.split('\\')[-2])
    images.append(image)
    labels.append(label)
end = time.time()
print("Finished preprocessing!")

print("\nTime spent preprocessing: {} seconds".format(round(end - start)))

Total training images:	39209
Processing image	35962 | 91.72% complete

In [ ]:
print(images[0:10])
print(labels[0:10])